In [39]:
# Import Libraries
import random
import nltk
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [1]:
# Read excel data
exl_data_df1 = pd.read_excel("C:\\NotBackedUp\\Code_Breaches_JuneDec2019.xlsx")
exl_data_df1.head()

Code chapter breached -1 Code chapter breached -2  \
0                   43 When we are recovering a debt                      NaN   
1                                  35 Joint Accounts                      NaN   
2  14 Taking extra care with customers who may be...                      NaN   
3  14 Taking extra care with customers who may be...                      NaN   
4                     04 Trained and competent staff                      NaN   

  Code chapter breached -3  Code chapter breached -4 ANZ indentifier   
0                      NaN                       NaN      COM-933838   
1                      NaN                       NaN       COM-920773  
2                      NaN                       NaN       COM-912862  
3                      NaN                       NaN       COM-905156  
4                      NaN                       NaN       COM-907041

In [2]:
# Read data from Teradata
teradata_df2 = pd.read_excel("C:\\NotBackedUp\\Historical_BCOP.xlsx")
teradata_df2.head()

Unnamed: 0    CNCRN_ID     RECVD_D  \
0           0  COM-904520  2019-07-01   
1           5  COM-904275  2019-07-01   
2          10  COM-923496  2019-10-03   
3          15  COM-904844  2019-07-02   
4          16  COM-934798  2019-11-27   

                                            CNCRN_DS  
0            credit reporting of the overdue account  
1  customer is not happy that he has not received...  
2  Customer required new pin and crn number, no c...  
3  Details of suggestion I have been a customer i...  
4  Customer is dissatisfied with low deposit rate...

In [3]:
# Outer join on Concern ID
master_df = pd.merge(teradata_df2, exl_data_df1, left_on='CNCRN_ID', right_on='ANZ indentifier ', how='outer')
master_df.head()

Unnamed: 0    CNCRN_ID     RECVD_D  \
0         0.0  COM-904520  2019-07-01   
1         5.0  COM-904275  2019-07-01   
2        10.0  COM-923496  2019-10-03   
3        15.0  COM-904844  2019-07-02   
4        16.0  COM-934798  2019-11-27   

                                            CNCRN_DS Code chapter breached -1  \
0            credit reporting of the overdue account                      NaN   
1  customer is not happy that he has not received...                      NaN   
2  Customer required new pin and crn number, no c...                      NaN   
3  Details of suggestion I have been a customer i...                      NaN   
4  Customer is dissatisfied with low deposit rate...                      NaN   

  Code chapter breached -2 Code chapter breached -3  Code chapter breached -4  \
0                      NaN                      NaN                       NaN   
1                      NaN                      NaN                       NaN   
2                      NaN                      NaN                       NaN   
3                      NaN                      NaN                       NaN   
4                      NaN                      NaN                       NaN   

  ANZ indentifier   
0              NaN  
1              NaN  
2              NaN  
3              NaN  
4              NaN

In [4]:
# Evaluate all columns in the merged dataframe
master_df.columns

Index(['Unnamed: 0', 'CNCRN_ID', 'RECVD_D', 'CNCRN_DS',
       'Code chapter breached -1', 'Code chapter breached -2',
       'Code chapter breached -3', 'Code chapter breached -4',
       'ANZ indentifier '],
      dtype='object')

In [5]:
# Drop the duplicate column from merged dataframe
# master_df = master_df.drop(["ANZ indentifier "], axis=1)
# master_df.tail()

In [6]:
# Dimension of merged dataframe
master_df.shape

(35741, 9)

### <font color = brown> 1. Ensure that there are no duplicate complaints on your data set </font>

In [7]:
# Ensure that there are no duplicate complaints on your data set
len(master_df["CNCRN_ID"].unique())
# Since unique values are only 35665 and total rows are 35741 - we have 35741-35665 duplicates = 76 duplicates

35665

In [8]:
# Select duplicate rows based Concern ID columns 
duplicateRowsDF = master_df[master_df.duplicated(['CNCRN_ID'])]
duplicateRowsDF.shape
# below shows 76 duplicate rows

(76, 9)

In [9]:
# dropping ALL duplicte 76 rows  
master_df.drop_duplicates(subset ="CNCRN_ID",keep ="first", inplace = True)
master_df.shape
# Received ID - fix

(35665, 9)

### <font color = brown> 2. Check if all the Complaints in excel have a match – those that doesn’t have a match please find the reason why are they dropped.

In [10]:
# Do a outer join on two dataframes using all rows from excel file
temp = pd.merge(teradata_df2, exl_data_df1, left_on='CNCRN_ID', right_on='ANZ indentifier ', how='right')
temp.shape

(314, 9)

In [11]:
temp_missing_df = temp.loc[ (temp["CNCRN_ID"] != temp['ANZ indentifier ']), :]
temp_missing_df.shape

(67, 9)

In [12]:
temp_missing_df.head(67)
# We see from below dataframe that 
# - Data in excel is not found in Teradata
# - Data is excel is not clean - has new line characters suffixed
# - One cell has multiple complaints id

Unnamed: 0 CNCRN_ID RECVD_D CNCRN_DS  \
0           NaN      NaN     NaN      NaN   
10          NaN      NaN     NaN      NaN   
12          NaN      NaN     NaN      NaN   
22          NaN      NaN     NaN      NaN   
38          NaN      NaN     NaN      NaN   
..          ...      ...     ...      ...   
226         NaN      NaN     NaN      NaN   
227         NaN      NaN     NaN      NaN   
258         NaN      NaN     NaN      NaN   
291         NaN      NaN     NaN      NaN   
309         NaN      NaN     NaN      NaN   

                              Code chapter breached -1  \
0                     43 When we are recovering a debt   
10   41 We will try to help you if you are experien...   
12   41 We will try to help you if you are experien...   
22             34 Direct debits and recurring payments   
38                     48 How we handle your complaint   
..                                                 ...   
226                   43 When we are recovering a debt   
227                47 If you have a complaint about us   
258                  08 Providing you with information   
291                  08 Providing you with information   
309                         42 When you are in default   

           Code chapter breached -2 Code chapter breached -3  \
0                               NaN                      NaN   
10                              NaN                      NaN   
12                              NaN                      NaN   
22                              NaN                      NaN   
38                              NaN                      NaN   
..                              ...                      ...   
226  04 Trained and competent staff                      NaN   
227                             NaN                      NaN   
258                             NaN                      NaN   
291                             NaN                      NaN   
309  04 Trained and competent staff                      NaN   

     Code chapter breached -4        ANZ indentifier   
0                         NaN             COM-933838   
10                        NaN            COM-906306\n  
12                        NaN            COM-913308\n  
22                        NaN            COM-908316\n  
38                        NaN             COM-908289   
..                        ...                     ...  
226                       NaN             COM-920192   
227                       NaN             COM-918684   
258                       NaN             COM-919575   
291                       NaN            COM-930854\n  
309                       NaN  COM-935461\nCOM-936033  

[67 rows x 9 columns]

In [13]:
# Removing the whitespace at end of the field
exl_data_df1['ANZ indentifier '] = exl_data_df1['ANZ indentifier '].str.strip()

In [14]:
temp = pd.merge(teradata_df2, exl_data_df1, left_on='CNCRN_ID', right_on='ANZ indentifier ', how='right')
temp.shape

(314, 9)

In [15]:
temp_missing_df = temp.loc[ (temp["CNCRN_ID"] != temp['ANZ indentifier ']), :]
temp_missing_df.shape

(4, 9)

In [16]:
temp_missing_df

Unnamed: 0 CNCRN_ID RECVD_D CNCRN_DS  \
40          NaN      NaN     NaN      NaN   
90          NaN      NaN     NaN      NaN   
208         NaN      NaN     NaN      NaN   
309         NaN      NaN     NaN      NaN   

                              Code chapter breached -1  \
40   41 We will try to help you if you are experien...   
90                    43 When we are recovering a debt   
208                   43 When we are recovering a debt   
309                         42 When you are in default   

           Code chapter breached -2 Code chapter breached -3  \
40                              NaN                      NaN   
90                              NaN                      NaN   
208                             NaN                      NaN   
309  04 Trained and competent staff                      NaN   

     Code chapter breached -4                                 ANZ indentifier   
40                        NaN                          COM-908612 \nCOM-920340  
90                        NaN                           COM-904550\nCOM-939864  
208                       NaN  COM-929790 \nCOM-918722\nCOM-928113\nCOM-907051  
309                       NaN                           COM-935461\nCOM-936033

In [17]:
from itertools import chain
import numpy as np

# return list from series of comma-separated strings
def chainer(s):
    return list(chain.from_iterable(s.str.split('\n')))

# calculate lengths of splits
lens = temp_missing_df['ANZ indentifier '].str.split('\n').map(len)
print(lens)

# create new dataframe, repeating or chaining as appropriate
res = pd.DataFrame({'CNCRN_ID': np.repeat(temp_missing_df['CNCRN_ID'], lens),
                    'RECVD_D': np.repeat(temp_missing_df['RECVD_D'], lens),
                    'CNCRN_DS': np.repeat(temp_missing_df['CNCRN_DS'], lens),
                    'Code chapter breached -1': np.repeat(temp_missing_df['Code chapter breached -1'], lens),
                    'Code chapter breached -2': np.repeat(temp_missing_df['Code chapter breached -2'], lens),
                    'Code chapter breached -3': np.repeat(temp_missing_df['Code chapter breached -3'], lens),
                    'Code chapter breached -4': np.repeat(temp_missing_df['Code chapter breached -4'], lens),
                    'ANZ indentifier ': chainer(temp_missing_df['ANZ indentifier '])})

res

40     2
90     2
208    4
309    2
Name: ANZ indentifier , dtype: int64


CNCRN_ID RECVD_D CNCRN_DS  \
40       NaN     NaN      NaN   
40       NaN     NaN      NaN   
90       NaN     NaN      NaN   
90       NaN     NaN      NaN   
208      NaN     NaN      NaN   
208      NaN     NaN      NaN   
208      NaN     NaN      NaN   
208      NaN     NaN      NaN   
309      NaN     NaN      NaN   
309      NaN     NaN      NaN   

                              Code chapter breached -1  \
40   41 We will try to help you if you are experien...   
40   41 We will try to help you if you are experien...   
90                    43 When we are recovering a debt   
90                    43 When we are recovering a debt   
208                   43 When we are recovering a debt   
208                   43 When we are recovering a debt   
208                   43 When we are recovering a debt   
208                   43 When we are recovering a debt   
309                         42 When you are in default   
309                         42 When you are in default   

           Code chapter breached -2 Code chapter breached -3  \
40                              NaN                      NaN   
40                              NaN                      NaN   
90                              NaN                      NaN   
90                              NaN                      NaN   
208                             NaN                      NaN   
208                             NaN                      NaN   
208                             NaN                      NaN   
208                             NaN                      NaN   
309  04 Trained and competent staff                      NaN   
309  04 Trained and competent staff                      NaN   

     Code chapter breached -4 ANZ indentifier   
40                        NaN      COM-908612   
40                        NaN       COM-920340  
90                        NaN       COM-904550  
90                        NaN       COM-939864  
208                       NaN      COM-929790   
208                       NaN       COM-918722  
208                       NaN       COM-928113  
208                       NaN       COM-907051  
309                       NaN       COM-935461  
309                       NaN       COM-936033

### <font color = brown> 3. Get the ratio of BCOP complaints vs NON-BCOP

In [18]:
# We did outer join on Excel data and teradata data and removed duplicates - This has total count of 35665
Total_CON_ID = master_df.shape[0]
print("Total_CON_ID : ", Total_CON_ID)

Total_CON_ID :  35665


In [19]:
# BCOP = rows in master_df where CNCRN_ID = ANZ indentifier
# (master_df["CNCRN_ID"] == master_df["ANZ indentifier "])
temp = master_df.loc[(master_df["CNCRN_ID"] == master_df["ANZ indentifier "]), :]
# Total number of NON-BCOP concern -id 
BCOP = temp.shape[0]
print("BCOP : ", BCOP)

BCOP :  237


In [20]:
# Total number of NON-BCOP concern -id 
NON_BCOP = Total_CON_ID - BCOP
print("NON_BCOP : ", NON_BCOP)

NON_BCOP :  35428


In [21]:
# Ratio of BCOP complaints vs NON-BCOP
Ratio = BCOP/ NON_BCOP
print("Ratio :", Ratio)

Ratio : 0.0066896240261939705


### <font color = brown> 4. Create a Target variable by using the  Code chapter breached 1 variable and rename it as breach_type

In [22]:
master_df.head()

Unnamed: 0    CNCRN_ID     RECVD_D  \
0         0.0  COM-904520  2019-07-01   
1         5.0  COM-904275  2019-07-01   
2        10.0  COM-923496  2019-10-03   
3        15.0  COM-904844  2019-07-02   
4        16.0  COM-934798  2019-11-27   

                                            CNCRN_DS Code chapter breached -1  \
0            credit reporting of the overdue account                      NaN   
1  customer is not happy that he has not received...                      NaN   
2  Customer required new pin and crn number, no c...                      NaN   
3  Details of suggestion I have been a customer i...                      NaN   
4  Customer is dissatisfied with low deposit rate...                      NaN   

  Code chapter breached -2 Code chapter breached -3  Code chapter breached -4  \
0                      NaN                      NaN                       NaN   
1                      NaN                      NaN                       NaN   
2                      NaN                      NaN                       NaN   
3                      NaN                      NaN                       NaN   
4                      NaN                      NaN                       NaN   

  ANZ indentifier   
0              NaN  
1              NaN  
2              NaN  
3              NaN  
4              NaN

In [23]:
# Add the new column Breach_type
master_df["Breach_Type"] = master_df['Code chapter breached -1'].str[:2]
# Verify the data in the newly added column
temp = master_df.loc[(master_df["Code chapter breached -1"].isnull() == False), :]
temp.head()

Unnamed: 0    CNCRN_ID     RECVD_D  \
332       1171.0  COM-916203  2019-08-26   
965       3883.0  COM-932410  2019-11-15   
1077      4411.0  COM-921492  2019-09-23   
1344      5694.0  COM-910452  2019-08-01   
1659      7267.0  COM-921375  2019-09-20   

                                               CNCRN_DS  \
332   i am writing to let you know what occured just...   
965   Customer was transferred through to me by a so...   
1077  Customer visited the branch in Richmond and wa...   
1344  What has happened is his ANZ cared expired tod...   
1659  customer was on payment arrangement and reques...   

                               Code chapter breached -1  \
332                      04 Trained and competent staff   
965   14 Taking extra care with customers who may be...   
1077                  08 Providing you with information   
1344                      05 Protecting confidentiality   
1659                   43 When we are recovering a debt   

     Code chapter breached -2 Code chapter breached -3  \
332                       NaN                      NaN   
965                       NaN                      NaN   
1077                      NaN                      NaN   
1344                      NaN                      NaN   
1659                      NaN                      NaN   

      Code chapter breached -4 ANZ indentifier  Breach_Type  
332                        NaN       COM-916203          04  
965                        NaN       COM-932410          14  
1077                       NaN       COM-921492          08  
1344                       NaN       COM-910452          05  
1659                       NaN       COM-921375          43

In [24]:
temp.shape

(235, 10)

### <font color = brown> 5. Those with blank chapter breach –label them as TBD for breach_type

In [25]:
master_df.head()

Unnamed: 0    CNCRN_ID     RECVD_D  \
0         0.0  COM-904520  2019-07-01   
1         5.0  COM-904275  2019-07-01   
2        10.0  COM-923496  2019-10-03   
3        15.0  COM-904844  2019-07-02   
4        16.0  COM-934798  2019-11-27   

                                            CNCRN_DS Code chapter breached -1  \
0            credit reporting of the overdue account                      NaN   
1  customer is not happy that he has not received...                      NaN   
2  Customer required new pin and crn number, no c...                      NaN   
3  Details of suggestion I have been a customer i...                      NaN   
4  Customer is dissatisfied with low deposit rate...                      NaN   

  Code chapter breached -2 Code chapter breached -3  Code chapter breached -4  \
0                      NaN                      NaN                       NaN   
1                      NaN                      NaN                       NaN   
2                      NaN                      NaN                       NaN   
3                      NaN                      NaN                       NaN   
4                      NaN                      NaN                       NaN   

  ANZ indentifier  Breach_Type  
0              NaN         NaN  
1              NaN         NaN  
2              NaN         NaN  
3              NaN         NaN  
4              NaN         NaN

In [26]:
# Find count of all rows where chapter breach is not present
temp = master_df.loc[(master_df['Code chapter breached -1'].isnull() == True), :]
temp.shape
# We found 35430 rows where Chapter Breach is blank

(35430, 10)

In [27]:
# Populate the Breach type field with TBD
# df.loc[conditions, field_name] = expresiion
master_df.loc[master_df['Code chapter breached -1'].isnull() == True, "Breach_Type"] = "TBD"

In [28]:
master_df.head()

Unnamed: 0    CNCRN_ID     RECVD_D  \
0         0.0  COM-904520  2019-07-01   
1         5.0  COM-904275  2019-07-01   
2        10.0  COM-923496  2019-10-03   
3        15.0  COM-904844  2019-07-02   
4        16.0  COM-934798  2019-11-27   

                                            CNCRN_DS Code chapter breached -1  \
0            credit reporting of the overdue account                      NaN   
1  customer is not happy that he has not received...                      NaN   
2  Customer required new pin and crn number, no c...                      NaN   
3  Details of suggestion I have been a customer i...                      NaN   
4  Customer is dissatisfied with low deposit rate...                      NaN   

  Code chapter breached -2 Code chapter breached -3  Code chapter breached -4  \
0                      NaN                      NaN                       NaN   
1                      NaN                      NaN                       NaN   
2                      NaN                      NaN                       NaN   
3                      NaN                      NaN                       NaN   
4                      NaN                      NaN                       NaN   

  ANZ indentifier  Breach_Type  
0              NaN         TBD  
1              NaN         TBD  
2              NaN         TBD  
3              NaN         TBD  
4              NaN         TBD

### <font color = brown> 6. Tag all Non-BCOP breaches in the historical file as non-breaches on breach_type variable

In [29]:
# Extract all non-BCOP records
#temp = master_df.loc[(master_df["CNCRN_ID"] != master_df["ANZ indentifier "]), :]
#temp.shape

In [30]:
# Mark Breach type = NB for all non BCOP records
master_df.loc[master_df["CNCRN_ID"] != master_df["ANZ indentifier "], "Breach_Type"] = "NB"
master_df.head()              

Unnamed: 0    CNCRN_ID     RECVD_D  \
0         0.0  COM-904520  2019-07-01   
1         5.0  COM-904275  2019-07-01   
2        10.0  COM-923496  2019-10-03   
3        15.0  COM-904844  2019-07-02   
4        16.0  COM-934798  2019-11-27   

                                            CNCRN_DS Code chapter breached -1  \
0            credit reporting of the overdue account                      NaN   
1  customer is not happy that he has not received...                      NaN   
2  Customer required new pin and crn number, no c...                      NaN   
3  Details of suggestion I have been a customer i...                      NaN   
4  Customer is dissatisfied with low deposit rate...                      NaN   

  Code chapter breached -2 Code chapter breached -3  Code chapter breached -4  \
0                      NaN                      NaN                       NaN   
1                      NaN                      NaN                       NaN   
2                      NaN                      NaN                       NaN   
3                      NaN                      NaN                       NaN   
4                      NaN                      NaN                       NaN   

  ANZ indentifier  Breach_Type  
0              NaN          NB  
1              NaN          NB  
2              NaN          NB  
3              NaN          NB  
4              NaN          NB

In [31]:
len(master_df["Breach_Type"].unique())

21

In [32]:
model_df = master_df[["CNCRN_DS","Breach_Type"]]
model_df.shape

(35665, 2)

### <font color = brown> Pre-processing

In [80]:
model_df = master_df[["CNCRN_DS", "Breach_Type"]]
model_df.head()

CNCRN_DS Breach_Type
0            credit reporting of the overdue account          NB
1  customer is not happy that he has not received...          NB
2  Customer required new pin and crn number, no c...          NB
3  Details of suggestion I have been a customer i...          NB
4  Customer is dissatisfied with low deposit rate...          NB

In [81]:
## Converting the read dataset in to a list of tuples, each tuple(row) contianing the message and it's label
data_set = []
for index,row in model_df.iterrows():
    data_set.append((row['CNCRN_DS'], row['Breach_Type']))

In [82]:
print(data_set[:5])

[('credit reporting of the overdue account', 'NB'), ('customer is not happy that he has not received the bonus interest for his progress saver account for  the month of June as the PP was not set up  customer states that when he set up the account he set it up in such a way that the PP were set up for all three account s customer is not happy that the PP was set up for the other 2 accounts but not this account  customer wants feedback lodged and the PP set up and interest paid', 'NB'), ('Customer required new pin and crn number, no complaint.', 'NB'), ('Details of suggestion I have been a customer in the past and my 17 year old is currently. I have never done this before but its time. I think you stance on the rugby player and stating that his views are not in line with your view of the world smacks of the limp, snowflake PC crowd. Im sick to death of big companies thinking that they are so progressive. Im voteing with my money and decission. We are looking at ending my sons account wi

In [83]:
print(len(data_set))

35665


In [84]:
data_set = data_set[:500]
print(len(data_set))

500


In [85]:
## initialise the inbuilt Stemmer and the Lemmatizer
stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

In [86]:
def preprocess(document, stem=True):
    'changes document to lower case, removes stopwords and lemmatizes/stems the remainder of the sentence'

    # change sentence to lower case
    document = document.lower()

    # tokenize into words
    words = word_tokenize(document)

    # remove stop words
    words = [word for word in words if word not in stopwords.words("english")]

    if stem:
        words = [stemmer.stem(word) for word in words]
    else:
        words = [wordnet_lemmatizer.lemmatize(word, pos='v') for word in words]

    # join words to make sentence
    document = " ".join(words)

    return document

In [87]:
## - Performing the preprocessing steps on all complaints
messages_set = []
for (complaints, label) in data_set:
    #print(complaints)
    words_filtered = [e.lower() for e in preprocess(complaints, stem=False).split() if len(e) >= 3]
    messages_set.append((words_filtered, label))

In [88]:
print(messages_set[:5])

[(['credit', 'report', 'overdue', 'account'], 'NB'), (['customer', 'happy', 'receive', 'bonus', 'interest', 'progress', 'saver', 'account', 'month', 'june', 'set', 'customer', 'state', 'set', 'account', 'set', 'way', 'set', 'three', 'account', 'customer', 'happy', 'set', 'account', 'account', 'customer', 'want', 'feedback', 'lodge', 'set', 'interest', 'pay'], 'NB'), (['customer', 'require', 'new', 'pin', 'crn', 'number', 'complaint'], 'NB'), (['detail', 'suggestion', 'customer', 'past', 'year', 'old', 'currently', 'never', 'time', 'think', 'stance', 'rugby', 'player', 'state', 'view', 'line', 'view', 'world', 'smack', 'limp', 'snowflake', 'crowd', 'sick', 'death', 'big', 'company', 'think', 'progressive', 'vote', 'money', 'decission', 'look', 'end', 'sons', 'account', 'put', 'big', 'board', 'work', 'call', 'piss', 'silent', 'majority', 'action', 'old', 'young', 'different', 'background', 'drop', 'account', 'bank', 'none', 'fly', 'quantas', 'despite', 'rugby', 'fan', 'match', 'believe',

In [89]:
## - creating a single list of all words in the entire dataset for feature list creation

def get_words_in_messages(messages):
    all_words = []
    for (message, label) in messages:
      all_words.extend(message)
    return all_words

In [90]:
## - creating a final feature list using an intuitive FreqDist, to eliminate all the duplicate words
## Note : we can use the Frequency Distribution of the entire dataset to calculate Tf-Idf scores 

def get_word_features(wordlist):

    #print(wordlist[:10])
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

In [91]:
## - creating the word features for the entire dataset
word_features = get_word_features(get_words_in_messages(messages_set))
print(len(word_features))

2909


### <font color = brown> Preparing to create a train and test set

In [1]:
## - creating slicing index at 80% threshold
sliceIndex = int((len(messages_set)*.8))

NameError: name 'messages_set' is not defined

In [2]:
## - shuffle the pack to create a random and unbiased split of the dataset
random.shuffle(messages_set)

NameError: name 'random' is not defined

In [94]:
train_messages, test_messages = messages_set[:sliceIndex], messages_set[sliceIndex:]

In [95]:
print(len(train_messages))
print(len(test_messages))

400
100


### <font color = brown> Preparing to create feature maps for train and test data

In [98]:
## creating a LazyMap of feature
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [99]:
## - creating the feature map of train and test data

training_set = nltk.classify.apply_features(extract_features, train_messages)
testing_set = nltk.classify.apply_features(extract_features, test_messages)

In [100]:
print(training_set[:5])

[({'contains(credit)': False, 'contains(report)': False, 'contains(overdue)': False, 'contains(account)': False, 'contains(customer)': False, 'contains(happy)': False, 'contains(receive)': False, 'contains(bonus)': False, 'contains(interest)': False, 'contains(progress)': False, 'contains(saver)': False, 'contains(month)': False, 'contains(june)': False, 'contains(set)': False, 'contains(state)': False, 'contains(way)': False, 'contains(three)': False, 'contains(want)': False, 'contains(feedback)': False, 'contains(lodge)': False, 'contains(pay)': False, 'contains(require)': False, 'contains(new)': False, 'contains(pin)': False, 'contains(crn)': False, 'contains(number)': False, 'contains(complaint)': False, 'contains(detail)': False, 'contains(suggestion)': False, 'contains(past)': False, 'contains(year)': False, 'contains(old)': False, 'contains(currently)': False, 'contains(never)': False, 'contains(time)': False, 'contains(think)': False, 'contains(stance)': False, 'contains(rugby)

In [101]:
print('Training set size : ', len(training_set))
print('Test set size : ', len(testing_set))

Training set size :  400
Test set size :  100


### <font color = brown> Training

In [102]:
## Training the classifier with NaiveBayes algorithm
myClassifier = nltk.NaiveBayesClassifier.train(training_set)

### <font color = brown> Evaluation

In [103]:
## - Analyzing the accuracy of the test set
print(nltk.classify.accuracy(myClassifier, training_set))

0.9975


In [104]:
## Analyzing the accuracy of the test set
print(nltk.classify.accuracy(myClassifier, testing_set))

1.0
